In [63]:
%run -i helper_functions.py

In [64]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from pathlib import Path
import yfinance as yf
from helper_functions import *
from dotenv import load_dotenv
import json
%matplotlib inline

In [65]:
# Load .env enviroment variables
load_dotenv(r"C:\Users\dredm\Desktop\Daryl\Fintech_Bootcamp\Git_Repos\Project-1\.env")
eia_api_key = os.getenv("EIA_API_KEY")

# dates to get data 
#end_date = date.today()
#start_date = end_date -  relativedelta(years=11)
#start_date

start_date = "2010-01-01"
end_date = "2014-12-30"
print(f"Key type:{type(eia_api_key)}")


Key type:<class 'str'>


In [66]:
# Reading Temperature data and convert it to dataframe
#Data cleanup
# Reading whale returns
df_weather = weather_data(r"Data\Pittsburg_Area_Temp_2010-2014.csv")
df_weather.head()




,Maximum,Minimum,Average
Date,,,
2010-01-01,34,20,27.0
2010-01-02,20,8,14.0
2010-01-03,18,6,12.0
2010-01-04,23,16,19.5
2010-01-05,24,19,21.5


In [67]:
# Reading Natural gas data and convert it to dataframe
#Data cleanup
#describe()

df_historic = yfinance_tickers_data("NG=F", start_date, end_date)
#df_historic = df_historic.drop(["Open", "High", "Low", "Volume", "Dividends", "Stock Splits" ], axis=1)
df_historic

,Close
Date,
2010-01-04,5.884
2010-01-05,5.637
2010-01-06,6.009
2010-01-07,5.806
2010-01-08,5.749
...,...
2014-12-22,3.144
2014-12-23,3.171
2014-12-26,3.007


In [ ]:
#Return close prices of Natural gas and Average temp 



In [68]:
#join close price of natural gas and Average temprature using concatenation

df = pd.concat((df_historic, df_weather), join="inner", axis=1 , sort=True).dropna()
display(df.isnull().sum())
df

Close      0
Maximum    0
Minimum    0
Average    0
dtype: int64

,Close,Maximum,Minimum,Average
Date,,,,
2010-01-04,5.884,23,16,19.5
2010-01-05,5.637,24,19,21.5
2010-01-06,6.009,26,22,24.0
2010-01-07,5.806,24,21,22.5
2010-01-08,5.749,23,11,17.0
...,...,...,...,...
2014-12-22,3.144,46,23,34.5
2014-12-23,3.171,53,39,46.0
2014-12-26,3.007,49,30,39.5


In [ ]:
#Find daily return of the two joined data frame
# Drop nulls
# Count nulls



In [ ]:
#plot daily return of the close price of naturan gas and temprature
#print out any similarities and differences between them
## Box plot to visually show risk



In [ ]:
# Calculate the daily standard deviations
# Calculate the annualized standard deviation (252 trading days)



In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation



In [ ]:
# Calculate and plot the correlation



In [ ]:
# explation the correlation between Temp and Price of Natural gas


In [69]:
# Linear regresssion with temp and natural gas prices
# Pull out average temp and natural gas closing price from the df
df_temp = df["Average"]
df_ng = df["Close"]
#drop the index from both so the arrays are 1-dimensional
df_temp.reset_index(drop=True, inplace=True)
df_ng.reset_index(drop=True, inplace=True)
#assign independent variable (x) and dependent variable (y) to the proper dataframe
#reshape the data frames to numpy arrays
x = np.array(df_temp).reshape(-1,1)
y = np.array(df_ng)
#print(X)
#create the model and fit it to the data
model = LinearRegression()
model.fit(x,y)
r_sq = model.score(x,y)
print("Coefficient of determination", r_sq)
print("Slope", model.coef_)
print("Intercept", model.intercept_)
y_pred = model.predict(x)
print("Predicted response:", y_pred, sep='\n')

Coefficient of determination 0.020098304306998327
Slope [-0.00562753]
Intercept 4.14313914625356
Predicted response:
[4.03340222 4.02214715 4.00807831 ... 3.92085152 3.97149934 3.99963701]


In [70]:
# Linear regresssion with percent change in temp and natural gas prices
#convert dataframe to percent change
df_pct_change = df.pct_change().dropna()
# Pull out average temp and natural gas closing price from the df
df_pct_chg_temp = df_pct_change["Average"]
df_pct_chg_ng = df_pct_change["Close"]
#drop the index from both so the arrays are 1-dimensional
df_pct_chg_temp.reset_index(drop=True, inplace=True)
df_pct_chg_ng.reset_index(drop=True, inplace=True)
#assign independent variable (x) and dependent variable (y) to the proper dataframe
#reshape the data frames to numpy arrays
x = np.array(df_pct_chg_temp).reshape(-1,1)
y = np.array(df_pct_chg_ng)
#print(x)
#create the model and fit it to the data
model = LinearRegression()
model.fit(x,y)
r_sq = model.score(x,y)
print("Coefficient of determination", r_sq)

Coefficient of determination 0.00821062445110221


In [71]:
#Group weather data by month, so we can use it with monthly consumption data
# Group by year and then month and get mean
df_avg_month_weather = df.groupby(by=[df.index.year, df.index.month]).mean()
#display(df_avg_price_weather.head())

#rename the new multi index
df_avg_month_weather.index.rename("Year", level=0, inplace = True)
df_avg_month_weather.index.rename("Month", level=1, inplace = True)

# Convert index to columns
df_avg_month_weather = pd.DataFrame(df_avg_month_weather.to_records()) 

# Add a 0 to months that are single digit
df_avg_month_weather["Month"] = df_avg_month_weather.Month.map("{:02}".format)

# Create a single new column to save the year and month
df_avg_month_weather['YearMonth'] = df_avg_month_weather['Year'].apply(str) + df_avg_month_weather['Month'].apply(str)

# Create an Date column to convert all dates with the first day of the month
df_avg_month_weather['DATE'] = pd.to_datetime(df_avg_month_weather["YearMonth"], format="%Y%m")

# Set index to the YearMonth column
df_avg_month_weather = df_avg_month_weather.set_index("YearMonth")

# Drop the year and month Columns
df_avg_month_weather = df_avg_month_weather.drop(["Year", "Month"], axis=1)
#Verify data
df_avg_month_weather.head()

,Close,Maximum,Minimum,Average,DATE
YearMonth,,,,,
201001,5.598895,31.368421,20.263158,25.815789,2010-01-01
201002,5.214000,33.210526,21.789474,27.500000,2010-02-01
201003,4.300000,53.826087,32.826087,43.326087,2010-03-01
201004,4.083619,67.666667,42.333333,55.000000,2010-04-01
201005,4.154050,74.100000,51.800000,62.950000,2010-05-01


In [72]:
#Get consumption data
# Pennsylvania Natural Gas Residential Consumption, Monthly
series_id = "NG.N3010PA2.M"
# get data of comsumption from EIA
data = eia_consumption_data_by_series(eia_api_key, series_id);

# create a data frame from the series of date and prices
df_comsumption  = pd.DataFrame(list(data["series"][0]["data"]))

#Rename the columns from 0 & 1 to YearMonth and Consumption
df_comsumption.rename(columns={0: "YearMonth", 1: "Consumption"}, inplace = True)

#Create a date column to select relevant data
df_comsumption['DATE'] = pd.to_datetime(df_comsumption["YearMonth"], format="%Y%m")

# Set datetype as datetime
df_comsumption["DATE"].astype('datetime64', copy=False)

# create mask to select only dates in our range
mask = (df_comsumption['DATE'] >= start_date) & (df_comsumption['DATE'] <= end_date)

# Apply mask and get relevant data 
df_comsumption = df_comsumption.loc[mask]

# Sort values 
df_comsumption= df_comsumption.sort_values(by="DATE", ascending = True)

# Set Index
df_comsumption.set_index("YearMonth", inplace = True)

# Drop date 
df_comsumption.drop(columns="DATE", inplace = True)
# Verify data
df_comsumption.head()

,Consumption
YearMonth,
201001,43694
201002,39423
201003,27737
201004,13190
201005,8276


In [73]:
#Linear regression with weather and consumption
# Pull out average temp and natural gas closing price from the df
df_avg_month_temp = df_avg_month_weather["Average"]
df_consumption = df_comsumption["Consumption"]
#drop the index from both so the arrays are 1-dimensional
df_avg_month_temp.reset_index(drop=True, inplace=True)
df_consumption.reset_index(drop=True, inplace=True)
#assign independent variable (x) and dependent variable (y) to the proper dataframe
#reshape the data frames to numpy arrays
x = np.array(df_avg_month_temp).reshape(-1,1)
y = np.array(df_consumption)
#print(X)
#create the model and fit it to the data
model = LinearRegression()
model.fit(x,y)
r_sq = model.score(x,y)
print("Coefficient of determination", r_sq)
print("Slope", model.coef_)
print("Intercept", model.intercept_)
y_pred = model.predict(x)
print("Predicted response:", y_pred, sep='\n')

Coefficient of determination 0.9398534240916885
Slope [-877.68810456]
Intercept 64569.16476190821
Predicted response:
[41910.95343112 40432.74188661 26542.37362319 16296.31901131
  9318.69858008  3450.15675371 -1675.38979628  -738.80919986
  5638.67774169 18135.28456371 26499.44322677 42267.90792341
 43131.63280812 37152.96002221 30358.4088604  17020.41169757
 10507.7569884   3230.73472757 -3166.41470723   994.01771013
  7707.51398815 17090.41777257 23668.89908958 30652.78872155
 34442.52057301 31480.32322013 18869.54095421 19741.24482169
  6661.69731811  2922.02408473 -2968.93488371  1757.22475757
  8720.48484566 15690.29627245 30865.94154694 31124.62856723
 37841.47224458 37915.16284985 33893.96550766 17313.63931432
  7559.33287959  4271.99197888   706.90648752  2333.09916609
  8375.18386769 15380.47055438 29184.47591506 35386.03528541
 47433.34938724 42927.22386798 34832.2558861  19096.56201156
  9128.5328241   2169.71999511  2754.84539815  3716.122846
  8418.02340612 16296.31901131

In [ ]:
#Test the model on out of sample data
#Need to get out of sample temp data for Pittsburg
#Need to get out of sample Pennsylvania Natural Gas Residential Consumption, Monthly for the same time period
#Run the temp data through our model to get predicted consumption
#Plot predicted consumption vs. actual consumption


In [76]:
#Regression with weather and storage
#bring in storage data 
storage_path = (r"Data\NG_STOR_WKLY_S1_W_1.csv")
storage_df = pd.read_csv(storage_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
#print the df
#storage_df.head(10)

#group storage data by year and month, then get the sum for the month
df_sum_month_storage = storage_df.groupby(by=[storage_df.index.year, storage_df.index.month]).sum()
#get length of the grouped df
#len(df_sum_month_storage)

#only use the first 60 months
df_60_month_storage = df_sum_month_storage.iloc[0:60]
#len(df_60_month_storage)

#pull storage of lower 48 out of the df
national_storage_df = df_60_month_storage["48 States"]
#display(national_storage_df)

#drop the index from national storage so it's 1-dimensional
national_storage_df.reset_index(drop=True, inplace=True)
#national_storage_df.head()

#assign independent variable (x) and dependent variable (y) to the proper dataframe
#reshape the data frames to numpy arrays
#x2 = np.array(df_avg_month_temp).reshape(-1,1)
#y2 = np.array(national_storage_df)
#len(x2)

#create the model and fit it to the data
#model = LinearRegression()
#model.fit(x2,y2)
#r_sq2 = model.score(x2,y2)
#print("Coefficient of determination", r_sq2)
#print("Slope", model.coef_)
#print("Intercept", model.intercept_)

weath_stor_df = pd.concat([national_storage_df, ])

Coefficient of determination 0.00238466978522045
Slope [10.86553792]
Intercept 11271.750063045849


In [ ]:
#we need to find a relationship that impacts price

In [ ]:
'''
def weather_data(file_path):
    weather_path = Path(file_path)
    weather_df = pd.read_csv(
    weather_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
    weather_df = weather_df.sort_index()
    weather_df = weather_df.drop(["Departure", "HDD", "CDD", "Precipitation", "New Snow", "Snow Depth" ], axis=1)
    return weather_df
'''

In [ ]:
%run -i helper_functions.py